# SOSE HEAT BUDGET

In [ ]:
import xarray as xr
from matplotlib import pyplot as plt
import gcsfs
import dask
import dask.array as dsa
import numpy as np
import intake
%matplotlib inline

In [ ]:
#example
cat_url = 'https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml'
cat = intake.Catalog(cat_url)
ds = cat.atmosphere.gmet_v1.to_dask()

In [ ]:
ocean_url = 'https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean.yaml'
ocean_cat = intake.Catalog(ocean_url)
ds = ocean_cat["SOSE"].to_dask()
ds

##### Another way to load (but slower)
ds = xr.open_zarr(gcsfs.GCSMap('pangeo-data/SOSE'))

ds

In [ ]:
ocean_cat.walk(depth=5)

In [ ]:
coords = ds.coords.to_dataset().reset_coords() #ds split into coords
dsr = ds.reset_coords(drop=True) #ds dropping coords so it just holds data variables
dsr

In [ ]:
import xgcm
grid = xgcm.Grid(ds, periodic=('XC', 'YC'))
grid

# Heat budget for Weddell Gyre

In [ ]:
 dsr.THETA[:,-5,:,10].plot()

In [ ]:
#why does 'isel' take so long to compute with

In [ ]:
dsr.THETA

In [ ]:
sst_mean = dsr.THETA.isel(Z=0).mean(dim='time').load()

In [ ]:
surface_mask = (coords.hFacC[0]>0).load()

In [ ]:
left_lon = 298
upper_lat = -65
right_lon = 360
lower_lat =-80

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'facecolor': '0.5'})
(sst_mean.where(surface_mask)).plot(ax=ax, vmin=-2, vmax=10)
ax.plot([left_lon, right_lon], [upper_lat, upper_lat], color='w')
ax.plot([right_lon, right_lon], [lower_lat, upper_lat], color='w')
ax.plot([left_lon, left_lon], [lower_lat, upper_lat], color='w')
ax.plot([left_lon, right_lon], [lower_lat, lower_lat], color='w')
#ax.patch(xy=(-80, 290), width=70, height=20)
#ax.patch(xy=(295,-80), width = 1, height = 1)
ax.set_xlim([287, 362])
ax.set_ylim([-80, -55])
plt.title('Surface Mask @ surface\nLat[290,360]\nLon[-80,-63.5]');

In [ ]:
#set the boundaries
lat_range = dict(YC=slice(lower_lat, upper_lat))
lon_range = dict(XC=slice(left_lon, right_lon))

In [ ]:
dsr.ADVy_TH.sel(YG=upper_lat, method='nearest')

In [ ]:
#advective flux
adv_flux_y = dsr.ADVy_TH.sel(**lon_range).sel(YG=upper_lat, method='nearest').sum(dim=['XC', 'Z']).load()
adv_flux_x = dsr.ADVx_TH.sel(**lat_range).sel(XG=right_lon, method='nearest').sum(dim=['YC', 'Z']).load()
adv_flux = adv_flux_x + adv_flux_y
adv_flux.load()
#units: (˚C m^3)/s

In [ ]:
yg_index = dsr.indexes['YG']
yg_index.get_loc(-60, method='nearest')

In [ ]:
adv_flux_y

In [ ]:
#diffusive flux
diff_flux_y = dsr.DFyE_TH.sel(**lon_range).sel(YG=upper_lat, method='nearest').sum(dim=['XC', 'Z']).load()
diff_flux_x = dsr.DFxE_TH.sel(**lat_range).sel(XG=360, method='nearest').sum(dim=['YC', 'Z']).load()
diff_flux = diff_flux_x + diff_flux_y
diff_flux.load()
#units: (˚C m^3)/s

In [ ]:
#vertical flux
t_flux_z = (dsr.TFLUX.sel(**lat_range, **lon_range) * coords.rA).sum(dim=['XC','YC'])
t_flux_z.load()
#units: W = (kg*m^2)/s^3

In [ ]:
coords.rA.sel(YC=-63.5, method='nearest')

In [ ]:
dsr.WTHMASS.sel(YC=-63.4583, method='nearest')

In [ ]:
#linear free surface correction
heat_capacity_cp = 3.994e3 #J/kg*˚C
runit2mass = 1.035e3 #kg/m^3
surface_term = t_flux_z / (heat_capacity_cp * runit2mass)
#units: (˚C m^3)/s

lin_fs_correction = -(dsr.WTHMASS.isel(Zl=0, drop=True) * coords.rA
                     ).sel(YC=slice(lower_lat, upper_lat)).sum(dim=['XC', 'YC'])
#lin_fs_correction longitude window is slightly different than other terms
#units: (˚C m^3)/s

In [ ]:
surface_term.load()
lin_fs_correction.load()

In [ ]:
surface_term.plot()
lin_fs_correction.plot()

In [ ]:
#is it hFacC or hFacS
total_volume = (coords.rA + coords.drF + coords.hFacC).sum().load()
total_volume

In [ ]:
dsr.TOTTTEND.sel(YC=upper_lat, method='nearest')

In [ ]:
tottend_weddell = (dsr.TOTTTEND * coords.rA * coords.drF * coords.hFacC
          ).sel(**lon_range, YC=slice(lower_lat, upper_lat)).sum(dim=['XC', 'YC', 'Z']) / (24*60*60)
tottend_weddell.load()
#same longitudinal window as lin_fs_correction (63.541634)
#units: (˚C m^3)/s

In [ ]:
adv_flux_y.plot(label='meridional')
adv_flux_x.plot(label='zonal')
plt.legend();

In [ ]:
rhs = + adv_flux + diff_flux + lin_fs_correction + surface_term
lhs = tottend_weddell
rhs.plot()
lhs.plot()

In [ ]:
(rhs - lhs).plot()

In [ ]:
(rhs - lhs).mean().load()

In [ ]:
all_terms = xr.merge([tottend_weddell.rename('tottend'),
                      adv_flux.rename('adv_flux'),
                      diff_flux.rename('diff_flux'),
                      lin_fs_correction.rename('lin_fs'),
                      surface_term.rename('surface')])
all_terms

In [ ]:
all_terms.to_array().plot.line(x='time')

In [ ]:
df = all_terms.mean(dim='time').reset_coords(drop=True).to_array().to_dataframe(name='budget')
df

In [ ]:
rhs_minus_lhs = df.iloc[0] - df.iloc[1:].sum()
rhs_minus_lhs

In [ ]:
budget_diff = rhs.sum() + lhs.sum()
budget_diff.load()

In [ ]:
df.plot(kind='bar')

In [ ]:
tottend_weddell.plot(label='dT/dt')
adv_flux.plot(label='advective term')
surface_term.plot(label='surface term')
lin_fs_correction.plot(label='lin-surf-correction')
diff_flux.plot(label='diffusive term')
plt.legend();

In [ ]:
adv_flux_y.plot()

In [ ]:
(surface_term - tottend_weddell).plot()
adv_flux_y.plot(label='adv flux y')

In [ ]:
test = surface_term - tottend_weddell
test

In [ ]:
dsr.TOTTTEND